(create_package)=

# Creating packages

In [2]:
from enviPath_python import enviPath
from enviPath_python.objects import *

On this tutorial we will create a package and try to replicate the [1[(4-chlorophenyl)phenylmethyl]piperazine (CPP)](https://envipath.org/package/7932e576-03c7-4106-819d-fe80dc605b8a/pathway/f6419e83-a922-484b-b4dd-341879338308) Pathway.

We start by defining the enviPath class and using https://envipath.org/ as `INSTANCE_HOST`. In order to create Package we need to assign a `Group` to it, we browse over all groups available on enviPath and return the "Anonymous" group.

In [3]:
eP = enviPath("https://envipath.org/")
anonymous_group = [group for group in eP.get_groups() if "Anonymous" in group.get_name()][0]
pkg = eP.create_package(
    name="Test Package", group=anonymous_group,
    description="This package is created as a tutorial for enviPath-python documentation!")

pkg.get_id()

'https://envipath.org/package/6e8c63f4-d1ba-4a84-a90c-0852c85157d6'

The pathway in question has 3 Nodes and 2 Edges, we add all the necessary information for the nodes and edges on lists of dictionaries and then we will loop over them to add them in the correct order. When we create the Pathway, the label `root_node_only=True` indicates that we want to add the Pathway manually, indicating as `False` would trigger a Pathway prediction.

In [4]:
node_list = [{"smiles": "C1=CC=C(C=C1)C(C2=CC=C(C=C2)Cl)N3CCNCC3", 
              "name": "1[(4-chlorophenyl)phenylmethyl]piperazine", "confidence": 1},
             {"smiles": "CC(=O)N1CCN(CC1)C(C2=CC=CC=C2)C3=CC=C(C=C3)Cl", 
              "name": "CPP_TP_M329", "confidence": 3},
             {"smiles": "C1=CC=C(C=C1)C(C2=CC=C(C=C2)Cl)N3CCN(CC3)C(=O)CCC(=O)O", 
              "name": "CPP_TP_M387", "confidence": 3}]

edge_list = [{"smirks": "C1=CC=C(C=C1)C(C2=CC=C(C=C2)Cl)N3CCNCC3>>CC(=O)N1CCN(CC1)C(C2=CC=CC=C2)C3=CC=C(C=C3)Cl"}, 
             {"smirks": "C1=CC=C(C=C1)C(C2=CC=C(C=C2)Cl)N3CCNCC3>>C1=CC=C(C=C1)C(C2=CC=C(C=C2)Cl)N3CCN(CC3)C(=O)CCC(=O)O"}]

pw = Pathway.create(pkg, smiles=node_list[0]["smiles"], 
                    name=node_list[0]["name"], root_node_only=True)

for node_info in node_list:
    pw.add_node(smiles=node_info["smiles"],
                    name=node_info["name"])
for edge_info in edge_list:
    pw.add_edge(smirks=edge_info["smirks"])

Next we want to create the scenario associated with this Pathway, and to do that we have to create a list of all the relevant `AdditionalInformation` objects. This part might look a bit tedious, but can be easily automatized when your experimental data comes from a flat file.

In [5]:
additional_info_list = []
ai = AcidityAdditionalInformation()
ai.set_lowPh(8)
ai.set_highPh(8)
additional_info_list.append(ai)

ai = BiologicalTreatmentTechnologyAdditionalInformation()
ai.set_biologicaltreatmenttechnology("nitrification & denitrification & biological phosphorus removal")
additional_info_list.append(ai)

ai = BioreactorAdditionalInformation()
ai.set_bioreactortype("amber Schott Flask")
ai.set_bioreactorsize(100)
additional_info_list.append(ai)

ai = FinalCompoundConcentrationAdditionalInformation()
ai.set_finalcompoundconcentration(120)
additional_info_list.append(ai)

ai = OriginalSludgeAmountAdditionalInformation()
ai.set_originalsludgeamount(50)
additional_info_list.append(ai)

ai = InoculumSourceAdditionalInformation()
ai.set_inoculumsource("nitrification basin")
additional_info_list.append(ai)

ai = LocationAdditionalInformation()
ai.set_location("WWTP Duebendorf (ARA Neugut) Switzerland")
additional_info_list.append(ai)

ai = PurposeOfWWTPAdditionalInformation()
ai.set_purposeofwwtp("mixed WW (municipal & industrial)")
additional_info_list.append(ai)

ai = RedoxAdditionalInformation()
ai.set_redoxType("aerob")
additional_info_list.append(ai)

ai = SludgeRetentionTimeAdditionalInformation()
ai.set_sludgeretentiontimeType("sludge age")
ai.set_sludgeretentiontime(25)
additional_info_list.append(ai)

ai = SolventForCompoundSolutionAdditionalInformation()
ai.set_solventforcompoundsolution1("MeOH")
additional_info_list.append(ai)

ai = SourceOfLiquidMatrixAdditionalInformation()
ai.set_sourceofliquidmatrix("WWTP effluent")
additional_info_list.append(ai)

ai = TemperatureAdditionalInformation()
ai.set_temperatureMin(20)
ai.set_temperatureMax(20)
additional_info_list.append(ai)

ai = TSSAdditionInformation()
ai.set_ttsStart(3.4)
ai.set_ttsEnd(3.4)
additional_info_list.append(ai)

ai = TypeOfAerationAdditionalInformation()
ai.set_typeofaeration("shaking")
additional_info_list.append(ai)

ai = AerationTypeAdditionalInformation()
ai.set_aerationtype("shaking on circulatin shaking table at 160 rpm")
additional_info_list.append(ai)

ai = TypeOfAdditionAdditionalInformation()
ai.set_typeofaddition("spiking in solvent")
additional_info_list.append(ai)
additional_info_list

Eventually we create a main `Scenario`, where all the common information will be added to and create 3 `ReferringScenarioAdditionalInformation` that point to the main one. This will copy all the additional information objects from the main scenario and we will only have to change the `ConfidenceLevelAdditionalInformation` object and append those to the corresponding node of the pathway.

In [11]:
main_scenario = Scenario.create(pkg, additional_information=additional_info_list)

for node_info in node_list:
    node = [n for n in pw.get_nodes() if (node_info["smiles"] == n.get_smiles())][0]
    
    ai = ConfidenceLevelAdditionalInformation()
    ai.set_radioconfidence(node_info["confidence"])
    
    referral_scenario = Scenario.create(pkg, referring_scenario_id=main_scenario.get_id(), additional_information=[])
    referral_scenario.update_scenario(additional_information=[ai])
    node.add_scenario(referral_scenario)

AttributeError: 'Package' object has no attribute 'id'

Additionally one can review the amount of existing data on the given package with just a few lines of code. In this tutorial, we use [Plotly](https://plotly.com/python/) to visualize the package statistics.

In [9]:
import plotly.express as px
import pandas as pd

d = {}
d["Compounds"] = len(pkg.get_compounds())
d["Nodes"] = sum([len(pw.get_nodes()) for pw in pkg.get_pathways()])
d["Reactions"] = len(pkg.get_reactions())
d["Edges"] = sum([len(pw.get_edges()) for pw in pkg.get_pathways()])
d["Scenarios"] = len(pkg.get_scenarios())
d["Pathways"] = len(pkg.get_pathways())

data = pd.DataFrame(d.items(), index=range(len(d.keys())), columns=["Object", "Amount"])
fig = px.bar(data, x="Object", y="Amount", color="Object", title="Amount of enviPath objects on the created Package")
fig

To finish and data cleaness purposes, we delete the package. If you download this notebook, you can comment the line below to see the generated pathway, but please make sure to delete the package once you are finished with this tutorial.

In [10]:
pkg.delete()